In [1]:
weather = pd.read_csv("weather3_180703.csv")
weather['date'] = pd.to_datetime(weather["date"])
station = weather[weather['station_nbr'] == 11]

In [2]:
station['date'].count()

1035

In [3]:
ls = []
ls1 = []
for i in station.columns:
    if station[i].count() == 0:
        ls.append(i)
    if station[i].isna().sum() != 0:
        ls1.append(i)
ls, ls1

([], ['depart', 'sunrise', 'sunset', 'resultdir', 'daytime'])

In [4]:
# for i in station.columns:
#     count_null = station[i].isna().sum()
#     print(i, ":", count_null, "(",round((count_null / len(station) * 100),2),"%",")")

In [5]:
train = pd.read_csv("train.csv")
train.date = pd.to_datetime(train.date)
train.tail()

,date,store_nbr,item_nbr,units
4617595,2014-10-31,45,107,0
4617596,2014-10-31,45,108,0
4617597,2014-10-31,45,109,0
4617598,2014-10-31,45,110,0
4617599,2014-10-31,45,111,0


In [6]:
key = pd.read_csv("key.csv")
station = station.merge(key)

In [7]:
station = station.merge(train)
station.tail()

,station_nbr,date,tmax,tmin,tavg,depart,dewpoint,wetbulb,heat,cool,...,daytime,week7,weekend,code_change,rainY,otherY,nothing,store_nbr,item_nbr,units
202237,11,2014-10-31,75.0,50.0,62.5,-2.0,53.0,57.0,2.0,0.0,...,11.0,4,0,0,0,0,1,43,107,0
202238,11,2014-10-31,75.0,50.0,62.5,-2.0,53.0,57.0,2.0,0.0,...,11.0,4,0,0,0,0,1,43,108,0
202239,11,2014-10-31,75.0,50.0,62.5,-2.0,53.0,57.0,2.0,0.0,...,11.0,4,0,0,0,0,1,43,109,0
202240,11,2014-10-31,75.0,50.0,62.5,-2.0,53.0,57.0,2.0,0.0,...,11.0,4,0,0,0,0,1,43,110,0
202241,11,2014-10-31,75.0,50.0,62.5,-2.0,53.0,57.0,2.0,0.0,...,11.0,4,0,0,0,0,1,43,111,0


In [8]:
station['log1p_units'] = np.log1p(station.units)

In [9]:
target1 = station['units']
target2 = station['log1p_units']

station.drop(columns=['units','log1p_units'],inplace=True)
station.tail()

,station_nbr,date,tmax,tmin,tavg,depart,dewpoint,wetbulb,heat,cool,...,windchill,daytime,week7,weekend,code_change,rainY,otherY,nothing,store_nbr,item_nbr
202237,11,2014-10-31,75.0,50.0,62.5,-2.0,53.0,57.0,2.0,0.0,...,62.522269,11.0,4,0,0,0,0,1,43,107
202238,11,2014-10-31,75.0,50.0,62.5,-2.0,53.0,57.0,2.0,0.0,...,62.522269,11.0,4,0,0,0,0,1,43,108
202239,11,2014-10-31,75.0,50.0,62.5,-2.0,53.0,57.0,2.0,0.0,...,62.522269,11.0,4,0,0,0,0,1,43,109
202240,11,2014-10-31,75.0,50.0,62.5,-2.0,53.0,57.0,2.0,0.0,...,62.522269,11.0,4,0,0,0,0,1,43,110
202241,11,2014-10-31,75.0,50.0,62.5,-2.0,53.0,57.0,2.0,0.0,...,62.522269,11.0,4,0,0,0,0,1,43,111


In [10]:
df1 = pd.concat([station,target1], axis=1)
df2 = pd.concat([station,target2], axis=1)

In [11]:
df2.to_csv('station11.csv', sep=',', index=False)

In [12]:
if df2.columns.tolist() and ls:
    print('공통 변수 있음')
    print(df2.columns.tolist() and ls)
else:
    print('공통 변수 없음')

공통 변수 없음


In [18]:
df2.columns

Index(['station_nbr', 'date', 'tmax', 'tmin', 'tavg', 'depart', 'dewpoint',
       'wetbulb', 'heat', 'cool', 'sunrise', 'sunset', 'codesum', 'snowfall',
       'preciptotal', 'stnpressure', 'sealevel', 'resultspeed', 'resultdir',
       'avgspeed', 'year', 'month', 'day', 'relative_humility', 'windchill',
       'daytime', 'week7', 'weekend', 'code_change', 'rainY', 'otherY',
       'nothing', 'store_nbr', 'item_nbr', 'log1p_units'],
      dtype='object')

### 1. 변수변환: df2 (log1p_units)

In [19]:
all_var = 'log1p_units ~ scale(tmax) + scale(tmin) + scale(tavg) + scale(depart) + scale(dewpoint) + scale(wetbulb) + scale(heat) + scale(cool)\
                            + scale(sunrise) + scale(sunset) + scale(snowfall) + scale(preciptotal) + scale(stnpressure) + scale(sealevel) + scale(resultspeed)  \
                            + C(resultdir) + scale(avgspeed) + C(year) + C(month) + scale(relative_humility) + scale(windchill) + C(weekend) \
                            + C(rainY) + C(store_nbr) + C(item_nbr) + 0'
not_all_var = ''

In [22]:
model2 = sm.OLS.from_formula('log1p_units ~ scale(tmax) + scale(tmin) + scale(tavg) + scale(depart) + scale(dewpoint) + scale(wetbulb) + scale(heat) + scale(cool)\
                            + scale(sunrise) + scale(sunset) + scale(preciptotal) + scale(stnpressure) + scale(sealevel) + scale(resultspeed)  \
                            + C(resultdir) + scale(avgspeed) + C(year) + C(month) + scale(relative_humility) + scale(windchill) + C(weekend) \
                            + C(rainY) + C(store_nbr) + C(item_nbr) + 0', data = df2)
result2 = model2.fit()
print(result2.summary())

                            OLS Regression Results                            
Dep. Variable:            log1p_units   R-squared:                       0.931
Model:                            OLS   Adj. R-squared:                  0.931
Method:                 Least Squares   F-statistic:                 1.416e+04
Date:                Fri, 06 Jul 2018   Prob (F-statistic):               0.00
Time:                        10:42:46   Log-Likelihood:             1.3776e+05
No. Observations:              185592   AIC:                        -2.752e+05
Df Residuals:                  185414   BIC:                        -2.734e+05
Df Model:                         177                                         
Covariance Type:            nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
C(resultdir)[1.0]       

### 2. 변수변환 : df2 (log1p_units) + 아웃라이어 제거

In [23]:
# 아웃라이어 제거
# Cook's distance > 2 인 값 제거
influence = result2.get_influence()
cooks_d2, pvals = influence.cooks_distance
fox_cr = 4 / (len(df2) - 2)
idx_outlier = np.where(cooks_d2 > fox_cr)[0]
len(idx_outlier)

4571

In [24]:
idx = list(set(range(len(df2))).difference(idx_outlier))
df2_1 = df2.iloc[idx, :].reset_index(drop=True)
df2_1

,station_nbr,date,tmax,tmin,tavg,depart,dewpoint,wetbulb,heat,cool,...,daytime,week7,weekend,code_change,rainY,otherY,nothing,store_nbr,item_nbr,log1p_units
0,11,2012-01-01,72.0,48.0,60.0,7.0,54.0,56.0,5.0,0.0,...,10.216667,6,1,1,1,0,0,12,1,0.000000
1,11,2012-01-01,72.0,48.0,60.0,7.0,54.0,56.0,5.0,0.0,...,10.216667,6,1,1,1,0,0,12,2,0.000000
2,11,2012-01-01,72.0,48.0,60.0,7.0,54.0,56.0,5.0,0.0,...,10.216667,6,1,1,1,0,0,12,3,0.000000
3,11,2012-01-01,72.0,48.0,60.0,7.0,54.0,56.0,5.0,0.0,...,10.216667,6,1,1,1,0,0,12,4,0.000000
4,11,2012-01-01,72.0,48.0,60.0,7.0,54.0,56.0,5.0,0.0,...,10.216667,6,1,1,1,0,0,12,5,0.000000
5,11,2012-01-01,72.0,48.0,60.0,7.0,54.0,56.0,5.0,0.0,...,10.216667,6,1,1,1,0,0,12,6,0.000000
6,11,2012-01-01,72.0,48.0,60.0,7.0,54.0,56.0,5.0,0.0,...,10.216667,6,1,1,1,0,0,12,7,0.000000
7,11,2012-01-01,72.0,48.0,60.0,7.0,54.0,56.0,5.0,0.0,...,10.216667,6,1,1,1,0,0,12,8,0.000000
8,11,2012-01-01,72.0,48.0,60.0,7.0,54.0,56.0,5.0,0.0,...,10.216667,6,1,1,1,0,0,12,9,0.000000
9,11,2012-01-01,72.0,48.0,60.0,7.0,54.0,56.0,5.0,0.0,...,10.216667,6,1,1,1,0,0,12,10,0.000000


In [25]:
# OLS - df2_1
model2_1 = sm.OLS.from_formula('log1p_units ~ scale(tmax) + scale(tmin) + scale(tavg) + scale(depart) + scale(dewpoint) + scale(wetbulb) + scale(heat) + scale(cool)\
                            + scale(sunrise) + scale(sunset) + scale(preciptotal) + scale(stnpressure) + scale(sealevel) + scale(resultspeed)  \
                            + C(resultdir) + scale(avgspeed) + C(year) + C(month) + scale(relative_humility) + scale(windchill) + C(weekend) \
                            + C(rainY) + C(store_nbr) + C(item_nbr) + 0', data = df2_1)
result2_1 = model2_1.fit()
print(result2_1.summary())

                            OLS Regression Results                            
Dep. Variable:            log1p_units   R-squared:                       0.998
Model:                            OLS   Adj. R-squared:                  0.998
Method:                 Least Squares   F-statistic:                 5.652e+05
Date:                Fri, 06 Jul 2018   Prob (F-statistic):               0.00
Time:                        10:44:58   Log-Likelihood:             5.5279e+05
No. Observations:              181021   AIC:                        -1.105e+06
Df Residuals:                  180843   BIC:                        -1.103e+06
Df Model:                         177                                         
Covariance Type:            nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
C(resultdir)[1.0]       

### 3. VIF 확인

In [26]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
# all_cols = ['tmax','tmin','tavg','dewpoint','wetbulb','heat','cool','preciptotal',\
#         'stnpressure','sealevel','resultspeed','avgspeed','relative_humility',\
#         'windchill', 'depart', 'sunrise', 'sunset', 'snowfall' ]
cols = ['tmax','tmin','tavg','dewpoint','wetbulb','heat','cool','preciptotal',\
        'stnpressure','sealevel','resultspeed','avgspeed','relative_humility',\
        'windchill', 'depart', 'sunrise', 'sunset', 'snowfall' ]

y = df2_1.loc[:,cols]
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(y.values, i) for i in range(y.shape[1])]
vif["features"] = y.columns
vif

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:167: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1388: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.uncentered_tss


,VIF Factor,features
0,inf,tmax
1,inf,tmin
2,inf,tavg
3,2.128507e+04,dewpoint
4,1.026768e+04,wetbulb
5,8.749700e+01,heat
6,2.515603e+02,cool
7,1.251613e+00,preciptotal
8,4.577464e+06,stnpressure
9,4.598409e+06,sealevel


### 4. VIF 근거, tmax, tmin, tavg 빼고 다시 Model 돌림.

In [27]:
# OLS - df2_1
model2_1 = sm.OLS.from_formula('log1p_units ~ scale(depart) + scale(dewpoint) + scale(wetbulb) + scale(heat) + scale(cool)\
                            + scale(sunrise) + scale(sunset) + scale(preciptotal) + scale(stnpressure) + scale(sealevel) + scale(resultspeed)  \
                            + C(resultdir) + scale(avgspeed) + C(year) + C(month) + scale(relative_humility) + scale(windchill) + C(weekend) \
                            + C(rainY) + C(store_nbr) + C(item_nbr) + 0', data = df2_1)
result2_1 = model2_1.fit()
print(result2_1.summary())

                            OLS Regression Results                            
Dep. Variable:            log1p_units   R-squared:                       0.998
Model:                            OLS   Adj. R-squared:                  0.998
Method:                 Least Squares   F-statistic:                 5.716e+05
Date:                Fri, 06 Jul 2018   Prob (F-statistic):               0.00
Time:                        10:55:19   Log-Likelihood:             5.5279e+05
No. Observations:              181021   AIC:                        -1.105e+06
Df Residuals:                  180845   BIC:                        -1.103e+06
Df Model:                         175                                         
Covariance Type:            nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
C(resultdir)[1.0]       

### 5. VIF 재확인 (tmax, tmin, tavg 제외함)

In [30]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
# all_cols = ['tmax','tmin','tavg','dewpoint','wetbulb','heat','cool','preciptotal',\
#         'stnpressure','sealevel','resultspeed','avgspeed','relative_humility',\
#         'windchill', 'depart', 'sunrise', 'sunset', 'snowfall' ]
cols = ['dewpoint','wetbulb','heat','cool','preciptotal',\
        'stnpressure','sealevel','resultspeed','avgspeed','relative_humility',\
        'windchill', 'depart', 'sunrise', 'sunset', 'snowfall' ]

y = df2_1.loc[:,cols]
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(y.values, i) for i in range(y.shape[1])]
vif["features"] = y.columns
vif = vif.sort_values("VIF Factor", ascending=False).reset_index(drop=True)
vif

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1388: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.uncentered_tss


,VIF Factor,features
0,4.500110e+06,stnpressure
1,4.486889e+06,sealevel
2,1.619230e+04,dewpoint
3,1.424655e+04,windchill
4,8.867807e+03,wetbulb
5,3.576182e+03,relative_humility
6,3.562436e+03,sunset
7,9.442026e+02,sunrise
8,2.330524e+02,cool
9,8.180873e+01,heat


### 6. VIF 근거, stnpressure, sealevel, dewpoint, windchill, wetbulb, relative_humility, sunset제외하고  Model 돌림.

In [32]:
# OLS - df2_1
model2_1 = sm.OLS.from_formula('log1p_units ~ scale(depart) + \
scale(heat) + scale(cool) + scale(sunrise) + \
scale(preciptotal) + scale(resultspeed) + \
C(resultdir) + scale(avgspeed) + C(year) + C(month) + \
C(weekend) + C(rainY) + C(store_nbr) + C(item_nbr) + 0', data = df2_1)
result2_1 = model2_1.fit()
print(result2_1.summary())

                            OLS Regression Results                            
Dep. Variable:            log1p_units   R-squared:                       0.998
Model:                            OLS   Adj. R-squared:                  0.998
Method:                 Least Squares   F-statistic:                 5.954e+05
Date:                Fri, 06 Jul 2018   Prob (F-statistic):               0.00
Time:                        11:05:26   Log-Likelihood:             5.5279e+05
No. Observations:              181021   AIC:                        -1.105e+06
Df Residuals:                  180852   BIC:                        -1.104e+06
Df Model:                         168                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
C(resultdir)[1.0]      0.0002      0

### 5. VIF 재확인 (stnpressure, sealevel, dewpoint, windchill, wetbulb, relative_humility, sunset제외)

In [33]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
# all_cols = ['tmax','tmin','tavg','dewpoint','wetbulb','heat','cool','preciptotal',\
#         'stnpressure','sealevel','resultspeed','avgspeed','relative_humility',\
#         'windchill', 'depart', 'sunrise', 'sunset', 'snowfall' ]
cols = ['heat','cool','preciptotal',\
        'resultspeed','avgspeed',\
        'depart', 'sunrise']

y = df2_1.loc[:,cols]
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(y.values, i) for i in range(y.shape[1])]
vif["features"] = y.columns
vif = vif.sort_values("VIF Factor", ascending=False).reset_index(drop=True)
vif

,VIF Factor,features
0,60.135406,avgspeed
1,40.134902,resultspeed
2,10.605508,sunrise
3,3.402939,heat
4,2.870536,cool
5,1.924635,depart
6,1.126055,preciptotal


### 6. VIF 근거, avgspeed, resultspeed, sunrise 제외하고  Model 돌림.

In [34]:
# OLS - df2_1
model2_1 = sm.OLS.from_formula('log1p_units ~ scale(depart) + \
scale(heat) + scale(cool) + \
scale(preciptotal) + \
C(resultdir) + C(year) + C(month) + \
C(weekend) + C(rainY) + C(store_nbr) + C(item_nbr) + 0', data = df2_1)
result2_1 = model2_1.fit()
print(result2_1.summary())

                            OLS Regression Results                            
Dep. Variable:            log1p_units   R-squared:                       0.998
Model:                            OLS   Adj. R-squared:                  0.998
Method:                 Least Squares   F-statistic:                 6.063e+05
Date:                Fri, 06 Jul 2018   Prob (F-statistic):               0.00
Time:                        11:07:59   Log-Likelihood:             5.5279e+05
No. Observations:              181021   AIC:                        -1.105e+06
Df Residuals:                  180855   BIC:                        -1.104e+06
Df Model:                         165                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
C(resultdir)[1.0]      0.0002      0

### Anova 분석을 통해 p-value가 0.05보다 큰것은 제외

In [35]:
anova_result2_1 = sm.stats.anova_lm(result2_1).sort_values(by=['PR(>F)'], ascending = False)
anova_result2_1[anova_result2_1['PR(>F)'] <= 0.05]

C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1821: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


,df,sum_sq,mean_sq,F,PR(>F)
C(rainY),1.0,0.124756,0.124756,956.252382,2.050262e-209
C(resultdir),36.0,95.647740,2.656882,20364.994533,0.000000e+00
C(year),2.0,0.205154,0.102577,786.251702,0.000000e+00
C(month),11.0,1.318285,0.119844,918.604643,0.000000e+00
C(weekend),1.0,0.964369,0.964369,7391.884733,0.000000e+00
C(store_nbr),1.0,1.312841,1.312841,10062.927628,0.000000e+00
C(item_nbr),110.0,13044.741649,118.588560,908981.164041,0.000000e+00


### 최종 Anova 분석과 TIF 통과한 변수들을 바탕으로 최종 모델생성

In [38]:
# OLS - df2_1_1
model2_1_1 = sm.OLS.from_formula('log1p_units ~ C(year) + C(month) + \
C(weekend) + C(rainY) + C(store_nbr) + C(item_nbr) + C(resultdir) + 0', data = df2_1)

result = model2_1_1.fit()
result2_1_1 = model2_1_1.fit()
print(result2_1_1.summary())

                            OLS Regression Results                            
Dep. Variable:            log1p_units   R-squared:                       0.998
Model:                            OLS   Adj. R-squared:                  0.998
Method:                 Least Squares   F-statistic:                 6.213e+05
Date:                Fri, 06 Jul 2018   Prob (F-statistic):               0.00
Time:                        11:15:10   Log-Likelihood:             5.5279e+05
No. Observations:              181021   AIC:                        -1.105e+06
Df Residuals:                  180859   BIC:                        -1.104e+06
Df Model:                         161                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
C(year)[2012]            0.0005 

In [48]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import KFold
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score


dfX = df2[['year', 'month', 'weekend', 'rainY', 'store_nbr', 'item_nbr', 'resultdir']]
dfy = df2['log1p_units']

model = LinearRegression()

cv = KFold(10, shuffle=True)

cross_val_score(model, dfX, dfy, scoring="r2", cv=cv)

array([0.01131646, 0.01475546, 0.01385164, 0.01462658, 0.01408325,
       0.01372394, 0.01375591, 0.01532111, 0.01476077, 0.01439645])

In [49]:
from patsy import dmatrix
# 독립변수와 종속변수로 나누기
df2_1_X = df2_1.drop(columns=['log1p_units'])
df2_1_target = df2_1['log1p_units']

In [50]:
formula = 'C(year) + C(month) + C(weekend) + C(resultdir) \
        + C(rainY) + C(store_nbr) + C(item_nbr) + 0'
dfX = dmatrix(formula, df2_1_X, return_type='dataframe')
dfy = pd.DataFrame(df2_1_target, columns=["log1p_units"])

In [51]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score

model = LinearRegression()
cv = KFold(10)

scores = np.zeros(10)
for i, (train_index, test_index) in enumerate(cv.split(dfX)):
    X_train = dfX.values[train_index]
    y_train = dfy.values[train_index]
    X_test = dfX.values[test_index]
    y_test = dfy.values[test_index]
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    scores[i] = r2_score(y_test, y_pred)

scores

array([0.99831571, 0.99829458, 0.99787071, 0.9980669 , 0.99849685,
       0.99794386, 0.99829532, 0.99826054, 0.99818238, 0.99813975])